In [71]:
print ("Testing")

Testing


In [80]:
%pwd

'c:\\'

In [77]:
import os
os.chdir("../")

In [82]:
%cd "C:/Coding Segment/Projects/Medical-Chatbot-Generative-AI"

C:\Coding Segment\Projects\Medical-Chatbot-Generative-AI


In [83]:
%pwd

'C:\\Coding Segment\\Projects\\Medical-Chatbot-Generative-AI'

In [84]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [85]:
def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents

In [86]:
extracted_data=load_pdf_file(data='Data/')

In [68]:
# extracted_data

In [87]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20
    )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [88]:
text_chunks = text_split(extracted_data)
print("Length of Text Chunks:", len(text_chunks))

Length of Text Chunks: 5859


In [16]:
# text_chunks

In [89]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [90]:
embeddings = download_hugging_face_embeddings()

In [91]:
query_result = embeddings.embed_query("Testing")
print("Length:", len(query_result))

Length: 384


In [26]:
# query_result

In [92]:
from dotenv import load_dotenv
load_dotenv()

True

In [93]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY = os.environ.get('GROQ_API_KEY')

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medi-bot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [40]:
import os
os.environ ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ ["GROQ_API_KEY"] = GROQ_API_KEY

In [ ]:
# Embed each chunk and upsert the embeddings into your Pinecone index.

from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
documents=text_chunks,
index_name=index_name,
embedding=embeddings,
)

In [33]:
#Load existing Index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [34]:
docsearch

In [35]:
retriever = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

In [36]:
retrived_docs = retriever.invoke("What is Acne?")

In [37]:
retrived_docs

[Document(id='429786bf-4e69-48f8-8451-71c963440e66', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 425.0, 'page_label': '426', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data\\THE GALE ENCYCLOPEDIA OF MEDICINE.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='Corticosteriod —A group of synthetic hormones\nthat are used to prevent or reduce inflammation.\nToxic effects may result from rapid withdrawal after\nprolonged use or from continued use of large doses.\nPatch test—A skin test that is done to identify aller-\ngens. A suspected substance is applied to the skin.\nAfter 24–48 hours, if the area is red and swollen,\nthe test is positive for that substance. If no reaction\noccurs, another substance is applied. This is con-'),
 Document(id='43f84203-c969-4dc0-a4d4-7d8a4e3bfdd7', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 

In [43]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature=0.4,
    model_name="llama3-70b-8192"
)

In [94]:
from langchain.chains import create_retrieval_chain 
from langchain.chains.combine_documents import create_stuff_documents_chain 
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, say that you don't know. "
    "Use three sentences maximum and keep the answer concise.\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system",system_prompt),
    ("human","{input}")
])

In [47]:
question_answer_chain = create_stuff_documents_chain (llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [48]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])

I don't know. The provided context does not mention acne.


In [49]:
response = rag_chain.invoke({"input": "What is Dysthymic disorder?"})
print(response["answer"])

Dysthymic disorder is a type of chronic depression that lasts two or more years, characterized by an underlying feeling of depression, and often accompanied by other psychiatric and physical conditions. It is a chronic condition that can lengthen or complicate the recovery of patients also suffering from medical conditions. Along with depression, individuals with dysthymic disorder may experience symptoms such as trouble sleeping, loss of interest in activities, weight changes, difficulty concentrating, and feelings of worthlessness and hopelessness.
